Imports

In [7]:
import requests
from bs4 import BeautifulSoup


1. Send reqest for accessing first webpage with opinious about progect

In [2]:
url="https://www.ceneo.pl/176439982;02514#tab=reviews_new"
respomse = requests.get(url)


2. If response is Ok, parse HTML page content into DOW sructure

In [8]:
if respomse.status_code ==200:
    page_dom = BeautifulSoup(respomse.text, 'html.parser')

3. Extract and their components from DOW opinions 